<a href="https://colab.research.google.com/github/Muazzam9/HelloWorld/blob/master/DSP_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import math
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.datasets import make_classification 
from sklearn.model_selection import train_test_split
from scipy.io import wavfile
import scipy as sp
import scipy.fftpack
import random
import wave, os, glob
from scipy.signal import hilbert, chirp
import numpy as np
import matplotlib.pyplot as plt 
from scipy.signal import find_peaks
import ntpath

In [ ]:
pip install pydub 

In [ ]:
pip install pysoundfile 

# Read File Functions

In [ ]:
def Read_Audio_Folder(r_folder_path):#folder path must be in raw string format
    audiodata = []
    audiosr = []
    files_paths= []
    for filename in glob.glob(os.path.join(r_folder_path,'*.wav')):
        
        files_paths.append(filename)
   
    files_paths = sorted(files_paths)#changed from str.lower
    return files_paths
# file_name = os.path.join(os.path.abspath('/UrbanSound8K/audio/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))

def file_name_finder(files_paths):#takes in a folder path and returns a list of the file names 
  file_names = []
  for i in range (len(files_paths)):#loops through array of file paths
    name = path_leaf(files_paths[i])
    file_names.append(name)
  return file_names#returns file names in the same order as the file paths with the extension

  import ntpath
def path_leaf(path):#function to rextract file name from a file path with the extension type or file type
    head, tail = ntpath.split(path)#splits the path based on the \ delimiter 
    return tail or ntpath.basename(head) #returns the filename with the extension

def key_name(file_names):
  key_names = []
  for i in range(len(file_names)):
    key = file_names[i].split('_',1)
    key_names.append(key[0])
  return key_names #returns an array of the key names

from statistics import mode
def dict_maker(mod_freq):
  keys = {}
  for i in range(0,len(mod_freq)):
    key = str(mod_freq[i][0])
    value = mod_freq[i][1]
    keys.update({key:value})
  return keys

def n_conv(arr):
  new_arr = []
  for i in arr:
    if i == 1:
      new_arr.append("A#5") 
    elif i == 2:
      new_arr.append("A5")
    elif i == 3:
      new_arr.append("B5")
    elif i == 4:
      new_arr.append("C#4")
    elif i == 5:
      new_arr.append("C4")
    elif i == 6:
      new_arr.append("C5")
    elif i == 7:
      new_arr.append("D#4")  
    elif i == 8:
      new_arr.append("D4")
    elif i == 9:
      new_arr.append("E4")
    elif i == 10:
      new_arr.append("F#4")
    elif i == 11:
      new_arr.append("F4")
    elif i == 12:
      new_arr.append("G#4")
    elif i == 13:
      new_arr.append("G4")  
  return new_arr

Machine Learning Functions

In [ ]:
def features(x,i):
  data, fs = librosa.load(x[i],sr=None, mono=True)
  fft = sp.fftpack.fft(data)
  sample_freq = sp.fftpack.fftfreq(data.size, d=1/fs)
  indices = (np.argsort(abs(fft))) #indices in ascending order of amplitudes
  reverse = indices[::-1] #indices in  decending order of amplitudes
  max_indices = reverse[0] #array of indicies relating to max amplitudes
  
  feature_list = (abs(sample_freq[max_indices]))
  return feature_list

def train(folder_path):
  y_train = []
  x_train = []
  x_train_file = []
  files_paths1 = Read_Audio_Folder('/content/drive/My Drive/Colab Notebooks/Audio')
  files_names1 = file_name_finder(files_paths1)
  a = 1
  j = 0 

  for i in range(130):
    x_train.append([features(files_paths1, i),i])
    x_train_file.append([features(files_paths1, i), files_names1[i]])
 
    if j == 10:
      j=0
      a=a+1 
    y_train.append(a)
    j=j+1
  return x_train, y_train, x_train_file

from statistics import mode
from collections import Counter
def mode_frequency(xtrainfile, keynames):
  mode_freq = []
  rxtrainfile = []
  for j in range(0,len(xtrainfile)):
    rounded_value = round(xtrainfile[j][0])
    rxtrainfile.append(rounded_value)
  for i in range(0,len(rxtrainfile),10):
    c = Counter(rxtrainfile[i:i+10])
    mod = np.asarray(c.most_common(1))
    mode_freq.append([keynames[i],mod[0][0]])
  return mode_freq

def calculateDistances(x_test, X_in):  
    distance_list = []
    for x in range(len(X_in)):
        distance_list.append(abs(x_test - X_in[x]))
    
    return distance_list

def kNearestIndices(distance_list, k):
    k_nearest_indices = (np.argsort(distance_list))[0:k]
    #print(type(k_nearest_indices))
    
    return k_nearest_indices

def kNearestNeighbours(k_nearest_indices, X_in, Y_in):

  """  X_k = (X_in[k_nearest_indices]) 
    Y_k = (Y_in[k_nearest_indices])
    D_k = (X_k,Y_k)
    #print(X_k.shape)
    return (X_k,Y_k)"""
  Y_k=[]
  X_k=[]
  for i in k_nearest_indices:{
      X_k.append(X_in[i]),Y_k.append(Y_in[i])
    }
  return X_k, Y_k

from scipy.stats import mode
def predict(x_test, X_in, Y_in, k):

    D_k = (kNearestNeighbours(kNearestIndices(calculateDistances(x_test, X_in), k), X_in, Y_in))
    prediction= mode(D_k[1])[0][0]
    #print(D_k[0])
    #print(D_k[1])
    return prediction

def predictBatch(X_t, X_in, Y_in, k):
    
    Y_k = []
    for i in X_t:
        Y_k.append((kNearestNeighbours(kNearestIndices(calculateDistances(i, X_in), k), X_in, Y_in))[1]) #dont need    
    #print(Y_k)
    #print(type(kNearestNeighbours(kNearestIndices(calculateDistances(i, X_in), k), X_in, Y_in)[1]))
    predictionslist=[]
    for a in Y_k:
        predictionslist.append(mode(a)[0][0])
        predictions = np.asarray(predictionslist)
    #print(type(predictions))
    #print(predictions.shape)
    return predictions

def accuracy(Y_pred, Y_test):
   
    count = 0
    for i in range(len(Y_test)):
        if Y_pred[i]==Y_test[i]:
            count = count+1
        else:
            count = count
    accuracy = count/len(Y_pred)
    
    return accuracy

def run(X_train, X_test, Y_train, Y_test, k):

    test_accuracy = accuracy(predictBatch(X_test, X_train, Y_train, k), Y_test)
    #print(predictBatch(X_test, X_train, Y_train, k))
    return test_accuracy

def createTrain(folder_path,no_samples_train):
  y_train = []
  x_train = []
  x_train_file = []
  files_paths1 = Read_Audio_Folder(folder_path)
  files_names1 = file_name_finder(files_paths1)
  a = 1
  j = 0 

  for i in range(len(files_paths1)):
    x_train.append(features(files_paths1,i))
    x_train_file.append([features(files_paths1, i), files_names1[i]])
 
    if j == no_samples_train:
      j=0
      a=a+1 
    y_train.append(a)
    j=j+1
  return x_train, y_train, x_train_file

def createTest(folder_path, no_tests_per_sample):
  y_test = []
  x_test = []
  x_test_file = []
  files_paths1 = Read_Audio_Folder(folder_path)
  files_names1 = file_name_finder(files_paths1)
  a = 1
  j = 0 

  for i in range(len(files_paths1)):
    x_test.append(features(files_paths1,i))
    x_test_file.append([features(files_paths1, i), files_names1[i]])
 
    if j == no_tests_per_sample:
      j=0
      a=a+1 
    y_test.append(a)
    j=j+1
  return x_test, x_test_file

def  data_list(audio_path):
  data_list1 = []
  data_ave = []
  a = 0
  data, fs = librosa.load(audio_path,sr=None)
  for i in data:
    data_list1.append(i)
  return(data_list1)

def  aveSig(audio_path,ave_amount):
  data_list = []
  data_ave = []
  a = 0
  data, fs = librosa.load(audio_path,sr=None)
  for i in data:
    data_list.append(i)
  while a < len(data_list):
    data_ave.append(sum(data_list[a:a + ave_amount])/ave_amount) 
    a = a + ave_amount
  print(len(data_list))
  print(len(data_ave))
  return(data_ave)

def thresh(data_ave, threshold, lowerlimit):
  thresh_data = []
  for i in range(len(data_ave)):
    if abs(data_ave[i]) >= threshold:
      thresh_data.append(data_ave[i])
    else:
      thresh_data.append(lowerlimit) 
  return(thresh_data)

def divide_chunks(list, size_chunk): 
    for i in range(0, len(list), size_chunk):  
        yield list[i:i + size_chunk] 

def LLToLNP(listlist):
  segment_arr = []
  for i in listlist:
    segment_arr.append(np.asarray(i))
  return(segment_arr)

def featuresForData(listOfArrays,minfreq,ave_amount):
  feature_list = []
  for x in listOfArrays:
    fft = sp.fftpack.fft(x)
    sample_freq = sp.fftpack.fftfreq(x.size, d=1/(44100/ave_amount))
    indices = (np.argsort(abs(fft))) #indices in ascending order of amplitudes
    reverse = indices[::-1] #indices in  decending order of amplitudes
    max_indices = reverse[0] #array of indicies relating to max amplitudes
    feature_list.append(abs(sample_freq[max_indices]))

  feature_list_reduced = []
  for z in feature_list:
    if z >= minfreq:
      feature_list_reduced.append(z)
  return(feature_list_reduced)

import math
def roundup10(f_list):
  feature_round = []
  for x in f_list:
    feature_round.append(roundup(x))
  return int(math.ceil(x / 10.0)) * 10

# DSP Functions

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
def chunk(song,silence_length,threshdB):
  chunks = split_on_silence (song, min_silence_len = silence_length,silence_thresh = threshdB,keep_silence=80)
  return(chunks)

def frequencies(chunks):
  frequency_list = []
  for i in chunks:
    wav_file = i
    samples = wav_file.get_array_of_samples()#gets the values instead of objects
    npsample = np.asarray(samples)
    fft = sp.fftpack.fft(npsample)
    sample_freq = sp.fftpack.fftfreq(npsample.size, d=1/(44100))
    indices = (np.argsort(abs(fft))) #indices in ascending order of amplitudes
    reverse = indices[::-1] #indices in  decending order of amplitudes
    max_indices = reverse[0] #array of indicies relating to max amplitudes
    frequency_list.append(abs(sample_freq[max_indices]))
  return(frequency_list)

def find_silence_length(song,thresh_dB,no_of_notes):#determines minimum silence length
  length = 0
  for i in range(200):#found that recommended silence ength is between 1 and 200ms
    chunks = chunk(song,i,thresh_dB)#chunks the data
    if(len(frequencies(chunks)) == no_of_notes):#checks if the number of frequencies found equals no. of notes
      length=i
      break
  return length#returns the silence length

def no_of_notes(data,fs,threshold):
  #data as a numpy array
  max_indexes = []
  sample_removal = int(round(0.2*fs))
  while ((data[int(np.argmax(data))])>threshold):
    current_max = int(np.argmax(data))
    max_indexes.append(current_max)
    for i in range(current_max-sample_removal,(current_max+(sample_removal+1))):
      data[i]=0
  maxes = np.asarray(max_indexes)
  return maxes, data
#staccato length = 0.812s to 0.963s = 0.15s this is the assumed minimum duration of a key press
#set the minimum peak to be 0.1

def find_first_index(envelope):
  for i in range(envelope.size):
    if envelope[i]>0.4:
      return i
      exit
    else:
      exit

def last_index_finder(start_index,envelope):
  for i in range(start_index,envelope.size):
    if envelope[i]<0.4:
      return i-1
      exit
    else:
      pass

def bpm_converter(bpm,fs):#function that takes in beats per minute
  bps = bpm/60 #converts beats per minute to beats per second
  beat_duration = (1/bps)*fs #the number of samples the beat would take up 
  return beat_duration

def squareSig(audio_file, threshold):
  data_pm, fs_pm = librosa.load(audio_file,sr=None)
  max1 = np.argmax(data_pm)
  scale = (0.8/data_pm[max1])
  data_pm = data_pm * scale
  split_list = []
  for i in range(len(data_pm)):
    if abs(data_pm[i]) >= threshold:
      split_list.append(data_pm[i])
    else:
      split_list.append(0) 
  slist = np.asarray(split_list)

  analytic_signal = hilbert(slist)
  amplitude_envelope = np.abs(analytic_signal)
  start_index = find_first_index(amplitude_envelope)
  end_index = last_index_finder(start_index,amplitude_envelope)
  arr = []
  for i in range(amplitude_envelope.size):
    if abs(amplitude_envelope[i])>threshold:
      arr.append(1.0)
    else: 
      arr.append(0)
  square_wave = np.asarray(arr)
  return(square_wave, data_pm, fs_pm)

def squareSig2(audio_file, threshold):
  song = AudioSegment.from_wav(audio_file)

  song = song.high_pass_filter(250)

  samples = song.get_array_of_samples()#gets the values instead of objects
  npsample = np.asarray(samples)
  data_pm = npsample
  fs_pm = 44100
  max1 = np.argmax(data_pm)
  scale = (0.8/data_pm[max1])
  data_pm = data_pm * scale
  split_list = []
  for i in range(len(data_pm)):
    if abs(data_pm[i]) >= threshold:
      split_list.append(data_pm[i])
    else:
      split_list.append(0) 
  slist = np.asarray(split_list)

  analytic_signal = hilbert(slist)
  amplitude_envelope = np.abs(analytic_signal)
  start_index = find_first_index(amplitude_envelope)
  end_index = last_index_finder(start_index,amplitude_envelope)
  arr = []
  for i in range(amplitude_envelope.size):
    if abs(amplitude_envelope[i])>threshold:
      arr.append(1.0)
    else: 
      arr.append(0)
  square_wave = np.asarray(arr)
  return(square_wave, data_pm, fs_pm)

def ref_square_wave(sq_wave,fs,bpm):
  min_duration = int(round(bpm_converter(bpm,fs)))
  counter = 0
  list_wave = []
  for x in sq_wave:
    list_wave.append(x)
  for i in range(len(list_wave)):
    if sq_wave[i]==1:
      counter = counter+1
      if counter ==1:
        for j in range(i+1,i+1+min_duration):
          sq_wave[j]=0
        counter=0
      else:
        sq_wave[i]=1
    else:
      sq_wave[i]=0
  return sq_wave

def one_counter(refine_wave):
  count = 0
  for i in refine_wave:
    if i==1:
      count = count+1
    else:
      pass 
  return count

def  aveSigArray(array,ave_amount):
  data_list = []
  data_ave1 = []
  a = 0
  data = np.asarray(array)
  for i in data:
    data_list.append(i)
  while a < len(data_list):
    data_ave1.append(sum(data_list[a:a + ave_amount])/ave_amount) 
    a = a + ave_amount
  return(np.asarray(data_ave1))

def silence(peaks,sample,average):
  durations=[]
  for i in range(len(peaks)-1):
    durations.append((peaks[i+1])/(sample/average)-(peaks[i])/(sample/average))
  return(durations)
def noteDuration(chunks,sample):
  duration_note = []
  for i in chunks: 
    samples = i.get_array_of_samples()#gets the values instead of objects
    npsample = np.asarray(samples)
    duration_note.append((len(npsample))/(sample))
  return(duration_note)


def enum(Y_test):
  for n, i in enumerate(Y_test):
    if i == 8:
      Y_test[n] = 6
  return Y_test

# Scripting Code

In [ ]:
def key_pos(keys,dif,time):#gets the position in array accorind to time and difference between notes.
  key_pos=[]
  times=[]
  val = 0
  for i in range(len(dif)):
    val = val+ dif[i]
    times.append(val)
  count=0

  b = int(time*2)


  for i in range(b+1):
    if(count >= len(times)):
      key_pos.append([0,0])
    elif(float(i/2)==times[count]):
      key_pos.append(keys[count])
      count+=1
    else:
      key_pos.append([0,0])
  if count < len(keys):
    key_pos.append(keys[count])
     
  return key_pos

def make_arr(key,dur): #makes an array of 2 length arrays
  arr=[]
  for i in range(len(key)):
    arr.append([key[i],dur[i]])
  return arr


def script(key,dif,dur):# main

  time = 0
  for i in range(len(dif)):        # standardizes the range of  durstions to 0.5 
     dif[i] = round(dif[i]*2) / 2
     if dif[i]==0:                 
       dif[i]=0.5
     dur[i] = round(dur[i]*2)/2
     if dif[i]==0:
       dif[i]=0.5
     time += dif[i] 
  for i in range(len(dur)): # standardizes the range of  durstions to 0.5 
     dur[i] = round(dur[i]*2) / 2
     if dur[i]==0:
      dur[i]=0.5
      dur[i] = round(dur[i]*2)/2
     if dur[i]==0:
      dur[i]=0.5

  no_sheets = math.ceil(time/10) #gets no of sheets
  key_dur = make_arr(key,dur) #gets 2d array of keys and dur
  #print(key_pos(key_dur,dif,time))
  position = key_pos(key_dur,dif,time) # gets the positions of spaces  
  length = len(position)
  count = 0
  sheet_count=0
  line1="___| "
  line2="   | "
  new_count=1
  print("Sheets                            Legend: 0.5s = ”*”, 1s = ”&”, 1.5s = ”$” , 2s = “+” "+"\n")
  for i in range(no_sheets):   # plots the number of sheets required
    for q in range(20):

      if new_count>=len(position): #checks if there is more notes to add
        line1 = line1 + "{0:>4}".format("_")
        line2 = line2 + "{0:>4}".format("-")
      elif position[new_count][0]!= 0:              #if not zero add number and duration
        line1 = line1 +  "{0:>4}".format(n_conv([position[new_count][0]])[0])
        if position[new_count][1]== 0.5:
          line2 = line2 + "{0:>4}".format("*")

        elif position[new_count][1] == 1:
          line2 = line2 + "{0:>4}".format("&")

        elif position[new_count][1] == 1.5:          # check the the symbol 
          line2 = line2 + "{0:>4}".format("$")

        elif position[new_count][1] == 2:
          line2 = line2 + "{0:>4}".format("+")
        new_count=new_count+1
      else:
        line1 = line1 + "{0:>4}".format("_")
        line2 = line2 + "{0:>4}".format("-")
        new_count = new_count + 1

      
        
    print('{:s}'.format('\u0332'.join(line1)))
    print("   |")
    print(line2)
    print("___|_________________________________________________________________________________"+"\n"+"\n")
    #print("____________________________________________________________________________________")
    line1 = "___| "
    line2 = "   | "

In [ ]:
def model(audio_path):
  square_wave,data_pm,fs_pm = squareSig(audio_path , 0.3)
  ave = 3980
  data_ave = aveSigArray(square_wave,ave)
  peaks2, _ = find_peaks(data_ave, prominence=0.1) 
  arr_len = len(peaks2)
  silences = silence(peaks2,fs_pm,ave)
  song = AudioSegment.from_wav(audio_path)
  song = song.high_pass_filter(250)
  length = find_silence_length(song,-16.6,arr_len)
  chunks = chunk(song,length,-16.6)
  freq_chunks = frequencies(chunks)
  X_test = freq_chunks
  predic = predictBatch(X_test, X_train, Y_train, k)
  note_durations = noteDuration(chunks,fs_pm)
  
  return(X_test,predic,note_durations,silences)

# Create Dataset

In [ ]:
X_train, Y_train, X_train_file = createTrain('/content/Audio',10)

# Model Test

In [ ]:
#Model
#Add folder
# Select perisan_market or  ode_joy.wav and set path equal to audi_path below:

audio_path = '/content/Single_Note_Songs.zip'

# Select Y_test by uncommenting required one for specific song:

#Persian Market
Y_test = [9,11,9,4,5,5,4,5,4,9,11,11,13,11,9,4,5,5,4,5,4,9,11] 

#Ode joy
#Y_test = [9,9,11,13,13,11,9,8,5,5,8,9,9,8,8,9,9,11,13,13,11,9,8,5,5,8,9,8,5,5,8,8,9,5,8,9,11,9,5,8,9,11,9,8,5,9,9,11,13,13,11,9,8,5,5,8,9,8,5,5] # ode joy 

#For corrected ode_joy uncomment code Y_test enum(Y_test)below, provided Y_test ode_joy is uncommented:
#Y_test = enum(Y_test)

k=2
X_test,key,dur, dif = model(audio_path)


predic = predictBatch(X_test, X_train, Y_train, k)
print('k =',k)
print("\n Test Batch\n",Y_test)
print("\n Predicted Batch\n",predic)
print("\nAccuracy = ",run(X_train, X_test, Y_train, Y_test, k))

print("\n"+"\n")
script(key,dif,dur)

k = 2

 Test Batch
 [9, 11, 9, 4, 5, 5, 4, 5, 4, 9, 11, 11, 13, 11, 9, 4, 5, 5, 4, 5, 4, 9, 11]

 Predicted Batch
 [ 9  9  5  4  5  5  4  5  4  9 11 11 13 11  9  4  5  5  4  5  4  9 11]

Accuracy =  0.9130434782608695



Sheets                            Legend: 0.5s = ”*”, 1s = ”&”, 1.5s = ”$” , 2s = “+” 

_̲_̲_̲|̲ ̲ ̲ ̲E̲4̲ ̲ ̲E̲4̲ ̲ ̲ ̲_̲ ̲ ̲C̲4̲ ̲C̲#̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲C̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲C̲4̲ ̲C̲#̲4̲ ̲ ̲C̲4̲ ̲C̲#̲4̲ ̲ ̲E̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲F̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_
   |
   |    *   *   -   *   *   -   -   *   -   -   *   *   *   *   *   -   -   *   -   -
___|_________________________________________________________________________________


_̲_̲_̲|̲ ̲ ̲ ̲F̲4̲ ̲ ̲G̲4̲ ̲ ̲F̲4̲ ̲ ̲ ̲_̲ ̲ ̲E̲4̲ ̲C̲#̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲C̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲C̲4̲ ̲C̲#̲4̲ ̲ ̲C̲4̲ ̲C̲#̲4̲ ̲ ̲E̲4̲ ̲ ̲F̲4̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_̲ ̲ ̲ ̲_
   |
   |    *   *   *   -   *   *   -   -   *   -   -   *   *   *   *   *   *   -   -   -
___|_____________________________________________________________________________